In [27]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

In [28]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
from keras.models import load_model
from lenet import LeNet

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os
import glob

# Import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [29]:
# use csv file to grab images/labels
csv_path = "../../temp_csvs/" + csv_file
df = pd.read_csv(csv_path)

#generate image path
lower_case_csv = csv_file.lower()
img_path = "../../images/" + lower_case_csv.split(".")[0]


# Choose column of label for prediction
models = os.listdir("./models/")

mod_menu = {}
for i in range(0, len(models)):
    mod_menu[models[i]] = models[i]

def f(models_menu):
    return models_menu

out2 = interact(f, models_menu=mod_menu)

interactive(children=(Dropdown(description='models_menu', options={'zaslavsk_cnn_Common Name_25_64.h5': 'zasla…

In [31]:
#select model
model = out2.widget.result

#get label
def getLabel(model):
    label_header_array1 = model.split("cnn_")
    label_header_array2 = label_header_array1[1].split("_")
    label_header = label_header_array2[0] 
    return label_header

labelHeader = getLabel(model)                   

In [32]:
# use csv file to grab images/labels
nameCol = df['#img']
predCol = df[label_header]

# init the image suffix, yset, and image list
suffix = '.jpg'
img_list = []

# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join(img_path, base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (28,28))
    im = img_to_array(im)
    img_list.append(im)

In [37]:
# Reshape original input data images for predicting
img_list = np.array(img_list, dtype ="float")/225.0


predictionsMade = []
preds = model.predict(img_list)

# Run all data through the prediction model that was created
for i in range (0,len(img_list)):
    predIndex = np.where(preds[i] == np.amax(preds[i]))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)
    
# Count how many correct predictions were made
correct = 0
for i in range (0,len(predictionsMade)):
    if(predictionsMade[i] == yset[i]):
        correct += 1 
        
print("Accuracy: " + str(correct/len(yset)))

[[[[5.15043439e-08 6.59411675e-08 6.90626429e-08]
   [5.03337906e-08 6.36000610e-08 6.75019052e-08]
   [5.15043439e-08 6.55509831e-08 6.94528273e-08]
   ...
   [1.01447950e-08 2.73129096e-08 4.52613931e-08]
   [1.05349794e-08 1.44368237e-08 1.95092212e-08]
   [1.36564548e-08 1.32662704e-08 1.48270081e-08]]

  [[5.15043439e-08 6.39902454e-08 6.82822740e-08]
   [5.03337906e-08 6.47706142e-08 6.82822740e-08]
   [4.48712087e-08 4.44810242e-08 4.87730529e-08]
   ...
   [1.05349794e-08 1.87288523e-08 3.04343850e-08]
   [3.90184423e-09 1.48270081e-08 2.26306965e-08]
   [5.07239750e-09 1.13153483e-08 1.59975613e-08]]

  [[5.07239750e-08 6.55509831e-08 6.94528273e-08]
   [4.25301021e-08 4.33104710e-08 4.64319463e-08]
   [2.18503277e-08 3.74577046e-08 4.44810242e-08]
   ...
   [1.52171925e-08 3.00442006e-08 3.31656760e-08]
   [3.90184423e-09 1.01447950e-08 1.75582990e-08]
   [5.07239750e-09 1.67779302e-08 1.79484835e-08]]

  ...

  [[2.34110654e-09 1.09251638e-08 1.83386679e-08]
   [2.73129096e-

AttributeError: 'str' object has no attribute 'predict'

In [35]:
preds

NameError: name 'preds' is not defined

In [ ]:
# Translate back to original csv label names
finalPred = []
for i in range (0,len(predictionsMade)):
    finalPred.append(yval_label[predictionsMade[i]])

from IPython.display import display
input_text = widgets.Text(
    value=None,
    placeholder='Type label here',
    disabled=False
)
output_text = widgets.Text(
    value=None,
    placeholder='New Header will be displayed here',
    disabled=False
)

def bind_input_to_output(sender):
    output_text.value = input_text.value

input_text.observe(bind_input_to_output)

print("Input new column Header Label: ")

display(input_text)
display(output_text)

In [ ]:
# Append the new column w/ it's new column name
df[input_text.value] = finalPred
print(input_text.value)

#Get file path
path = "../../temp_csvs"

# new file name
new_file =  csv_file[:-4]+'_v1.csv'
df.to_csv(os.path.join(path, new_file), index=None)

In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.observe(bind_input_to_output)

print("Input survey name here:")
# Display input text box widget for input
display(input_text)

display(output_text)

survey_name = output_text.value

In [ ]:
#Parse url
upload_url = survey_url.split("/main")[0]

if "https" in upload_url:
    upload_url = upload_url.replace("s","",1)
    upload_url = upload_url + ":3001/uploadCSV"    


new_survey_url_base = survey_url.split(user)[0]

In [ ]:
import requests
upload_data = {'name': input_text.value, 'dzc': dzc_file, 'user':user}
files = {"file": open(new_file, "rb")}
r = requests.post(upload_url, files=files, data=upload_data)
print(r.status_code, r.reason)

regex = re.compile('[^0-9a-zA-Z_]')
survey_url = survey_name
survey_url =  regex.sub('_', survey_url)

url = new_survey_url_base + user + "_" + input_text.value + ".csv" + "&views=" + views + "&view=" + view
print(url)
print ("Click the URL to open the new survey")
